# Pré-traitement du dataset recipes

Dataset issu de kaggle : https://www.kaggle.com/datasets/thedevastator/better-recipes-for-a-better-life?resource=download

In [1]:
import numpy as np
import pandas as pd
import json

def show_df(df):
    print(df.shape)
    display(pd.concat([df.head(2), df.tail(1)]))

In [ ]:
df = pd.read_csv("/home/camille/code/Camille9999/EPSI/workshop-m1/data/data/kaggle/recipes.csv")
show_df(df)

(1090, 15)


,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
1089,1089,Chef John's Chocolate Energy Bars,15 mins,NaN,3 hrs,12,12 bars,"2 cups pitted Medjool dates, roughly chopped, ...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,/Appetizers and Snacks/Snacks/Granola Bar Reci...,"Total Fat 22g 28%, Saturated Fat 7g 36%, Sodiu...","Prep Time: 15 mins, Additional Time: 2 hrs 45 ...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...


In [3]:
import uuid
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download();

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Parser ingrédients

In [4]:
'¼', '½', '¾', '⅓', '⅔', '⅛', '⅜', '⅝', '⅞'

('¼', '½', '¾', '⅓', '⅔', '⅛', '⅜', '⅝', '⅞')

In [5]:
def compact_numbers_in_string(s):
    def compact_match(match):
        numbers = match.group().split()
        total = sum(float(num) for num in numbers)
        return str(total)
    compacted_string = re.sub(r'(\d+\.\d+|\d+)(\s+(\d+\.\d+|\d+))+', compact_match, s)
    return compacted_string

def fraction_to_decimal(text):
    def replace_fraction(match):
        numerator, denominator = map(int, match.group().split('/'))
        decimal_value = round(numerator / denominator, 3)
        return f' {decimal_value} '
    fraction_pattern = re.compile(r' (?P<numerator>\d+)/(?P<denominator>\d+) ')
    return fraction_pattern.sub(replace_fraction, text)

def convert_fraction_to_decimal(fraction_str: str):
    unicode_fractions = {
        '¼': '0.25',
        '½': '0.5',
        '¾': '0.75',
        '⅓': '0.33',
        '⅔': '0.67',
        '⅛': '0.125',
        '⅜': '0.375',
        '⅝': '0.625',
        '⅞': '0.875'
    }
    for key, val in unicode_fractions.items():
        fraction_str = fraction_str.replace(key, val)
    fraction_str = compact_numbers_in_string(fraction_str)
    fraction_str = fraction_to_decimal(fraction_str)
    return fraction_str

def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

def split_ingredients(ingredient_list):
    return re.split(r',\s*(?=\d)', ingredient_list)

def convert_to_metric(amount, unit):
    conversion_factors = {
        'teaspoon': 4.92892,
        'teaspoons': 4.92892,
        'tablespoon': 14.7868,
        'tablespoons': 14.7868,
        'cup': 240,
        'cups': 240,
        'ounce': 28.3495,
        'ounces': 28.3495,
        'pint': 473.176,
        'pints': 473.176,
        'quart': 946.353,
        'quarts': 946.353,
        'gallon': 3785.41,
        'gallons': 3785.41,
        'pound': 453.592,
        'pounds': 453.592
    }

    if unit in conversion_factors:
        metric_amount = float(amount) * conversion_factors[unit]
        if unit in ['pound', 'pounds', 'ounce', 'ounces']:
            return round(metric_amount), 'g'
        else:
            return round(metric_amount), 'ml'
    return amount, None


def extract_ingredients(ingredient_list):
    ingredient_list = convert_fraction_to_decimal(ingredient_list)
    ingredient_list = remove_parentheses(ingredient_list)
    ingredient_parts = split_ingredients(ingredient_list)

    common_units = {'teaspoon', 'teaspoons', 'tablespoon', 'tablespoons', 'cup', 'cups',
                    'ounce', 'ounces', 'pint', 'pints', 'quart', 'quarts',
                    'gallon', 'gallons', 'pound', 'pounds'}

    ingredients = {}
    for part in ingredient_parts:
        tokens = word_tokenize(part)
        tagged = pos_tag(tokens)
        amount = convert_fraction_to_decimal(tagged[0][0])
        unit = None
        ingredient = []
        percentage = None
        for i, (word, tag) in enumerate(tagged):
            if tag in ['NN', 'NNS', 'JJ']:  # Noun or adjective
                if word.lower() in common_units:
                    unit = word
                elif word == '%':
                    percentage = f"{tagged[i-1][0]}%"
                else:
                    ingredient.append(word)
        if amount is not None and ingredient:
            ingredient_name = ' '.join(ingredient).strip()
            if percentage:
                ingredient_name = f"{percentage} {ingredient_name}"
            try:
                metric_amount, metric_unit = convert_to_metric(amount, unit)
            except Exception as e:
                print(f"Erreur de conversion: {e}\n - ingredient_list : {ingredient_list},\n - ingredient : {part}")
                metric_amount, metric_unit = amount, unit
            try:
                metric_amount = float(metric_amount)
            except:
                metric_amount = 1
            ingredients[ingredient_name] = {"amount": metric_amount, "unit": metric_unit}

    return ingredients

# Example ingredient lists
ingredient_list1 = "3 tablespoons butter, 2 pounds Granny Smith apples (or other firm, crisp apples), peeled, quartered, cored and sliced 1/4-inch thick, 1 pound Macintosh apples (or other soft-textured apples that fall apart when cooked), peeled, quartered, cored, and sliced 1/4-inch thick, ½ cup sugar, ½ cup dried cranberries, 1 sheet frozen puff pastry, thawed but still cold (follow package directions), 1 egg white, lightly beaten, 1 tablespoon sugar, 1 cup Optional: Ice cream or lightly sweetened whipped cream"
ingredient_list2 = "2 tablespoons butter, 3 skinless, boneless chicken breast halves, 1 ½ teaspoons salt, 1 ½ teaspoons ground black pepper, 2 tablespoons garlic powder, divided, 1 lemon, juiced"
ingredient_list3 = "1 shortcrust pastry crust, 6 ½ ounces 70% dark chocolate, chopped, 1 ½ cups ground almonds, 2 eggs, ½ cup superfine sugar, 6 ½ tablespoons butter, softened, 3 pears - peeled, cored, and quartered"

print(extract_ingredients(ingredient_list1))
print(extract_ingredients(ingredient_list2))
print(extract_ingredients(ingredient_list3))


{'butter': {'amount': 44.0, 'unit': 'ml'}, 'apples 1/4-inch thick': {'amount': 454.0, 'unit': 'g'}, 'sugar': {'amount': 15.0, 'unit': 'ml'}, 'cranberries': {'amount': 120.0, 'unit': 'ml'}, 'sheet frozen puff pastry': {'amount': 1.0, 'unit': None}, 'egg white': {'amount': 1.0, 'unit': None}, 'cream cream': {'amount': 240.0, 'unit': 'ml'}}
{'butter': {'amount': 30.0, 'unit': 'ml'}, 'skinless boneless chicken breast halves': {'amount': 3.0, 'unit': None}, 'salt': {'amount': 7.0, 'unit': 'ml'}, 'ground black pepper': {'amount': 7.0, 'unit': 'ml'}, 'garlic powder': {'amount': 30.0, 'unit': 'ml'}, 'lemon': {'amount': 1.0, 'unit': None}}
{'shortcrust pastry crust': {'amount': 1.0, 'unit': None}, '70% dark chocolate': {'amount': 184.0, 'unit': 'g'}, 'ground almonds': {'amount': 360.0, 'unit': 'ml'}, 'eggs': {'amount': 2.0, 'unit': None}, 'superfine sugar': {'amount': 120.0, 'unit': 'ml'}, 'butter': {'amount': 96.0, 'unit': 'ml'}, 'pears': {'amount': 3.0, 'unit': None}}


## Parser nutrition

In [6]:
def calculate_calories(nutrients):
    calories_per_gram = {
        'Total Fat': 9,
        'Saturated Fat': 9,
        'Total Carbohydrate': 4,
        'Protein': 4
    }
    total_calories = 0
    for nutrient, details in nutrients.items():
        if nutrient in calories_per_gram:
            amount_in_grams = int(details['amount'].replace('g', ''))
            total_calories += amount_in_grams * calories_per_gram[nutrient]
    percent = round(100 * total_calories / 2500)
    return total_calories, percent


def extract_nutrition(nutritional_values):
    entries = nutritional_values.split(', ')
    nutrition_dict = {}
    for entry in entries:
        parts = entry.split(' ')
        name = ' '.join(parts[:-2]) if '%' in parts[-1] else ' '.join(parts[:-1])
        amount = parts[-2] if '%' in parts[-1] else parts[-1]
        percent = parts[-1] if '%' in parts[-1] else None
        nutrition_dict[name] = {'amount': amount, 'percent': percent}
    total_calories, percent = calculate_calories(nutrition_dict)
    nutrition_dict['Calories'] = {'amount': f"{total_calories}kcal", 'percent': f"{percent}%"}
    return nutrition_dict


nutritional_values = 'Total Fat 0g 0%, Sodium 24mg 1%, Total Carbohydrate 41g 15%, Dietary Fiber 2g 5%, Total Sugars 22g, Protein 1g, Vitamin C 30mg 148%, Calcium 23mg 2%, Iron 1mg 3%, Potassium 228mg 5%'
print(extract_nutrition(nutritional_values))

nutritional_values2 = 'Total Fat 19g 24%, Saturated Fat 9g 46%, Cholesterol 31mg 10%, Sodium 124mg 5%, Total Carbohydrate 52g 19%, Dietary Fiber 3g 10%, Protein 2g, Vitamin C 5mg 27%, Calcium 23mg 2%, Iron 1mg 6%, Potassium 156mg 3%'
print(extract_nutrition(nutritional_values2))


{'Total Fat': {'amount': '0g', 'percent': '0%'}, 'Sodium': {'amount': '24mg', 'percent': '1%'}, 'Total Carbohydrate': {'amount': '41g', 'percent': '15%'}, 'Dietary Fiber': {'amount': '2g', 'percent': '5%'}, 'Total Sugars': {'amount': '22g', 'percent': None}, 'Protein': {'amount': '1g', 'percent': None}, 'Vitamin C': {'amount': '30mg', 'percent': '148%'}, 'Calcium': {'amount': '23mg', 'percent': '2%'}, 'Iron': {'amount': '1mg', 'percent': '3%'}, 'Potassium': {'amount': '228mg', 'percent': '5%'}, 'Calories': {'amount': '168kcal', 'percent': '7%'}}
{'Total Fat': {'amount': '19g', 'percent': '24%'}, 'Saturated Fat': {'amount': '9g', 'percent': '46%'}, 'Cholesterol': {'amount': '31mg', 'percent': '10%'}, 'Sodium': {'amount': '124mg', 'percent': '5%'}, 'Total Carbohydrate': {'amount': '52g', 'percent': '19%'}, 'Dietary Fiber': {'amount': '3g', 'percent': '10%'}, 'Protein': {'amount': '2g', 'percent': None}, 'Vitamin C': {'amount': '5mg', 'percent': '27%'}, 'Calcium': {'amount': '23mg', 'perc

## Parser time

In [7]:
def extract_time(time_str):
    parts = time_str.split()
    total_minutes = 0
    for i in range(0, len(parts), 2):
        value = int(parts[i])
        unit = parts[i+1]
        if unit.startswith('hr'):
            total_minutes += value * 60
        elif unit.startswith('min'):
            total_minutes += value
    return total_minutes

time1 = "1 hrs 30 mins"
time2 = "25 mins"
extract_time(time1), extract_time(time2)

(90, 25)

## Formating directions

In [8]:
def format_directions(string: str):
    string = string.split('\n\n')[0]
    return string

direction1 = df.at[1, 'directions']
direction2 = df.at[2, 'directions']

print(format_directions(direction1), '\n')
print(format_directions(direction2))

Peel and core apples, then thinly slice. Set aside.
Preheat the oven to 425 degrees F (220 degrees C).
Melt butter in a saucepan over medium heat. Add flour and stir to form a paste; cook until fragrant, about 1 to 2 minutes. Add both sugars and water; bring to a boil. Reduce the heat to low and simmer for 3 to 5 minutes. Remove from the heat.
Press one pastry into the bottom and up the sides of a 9-inch pie pan. Roll out remaining pastry so it will overhang the pie by about 1/2 inch. Cut pastry into eight 1-inch strips.
Place sliced apples into the bottom crust, forming a slight mound. Lay four pastry strips vertically and evenly spaced over apples, using longer strips in the center and shorter strips at the edges.
Make a lattice crust: Fold the first and third strips all the way back so they're almost falling off the pie. Lay one of the unused strips perpendicularly over the second and fourth strips, then unfold the first and third strips back into their original position.
Fold the s

## Cleaning du dataset

In [9]:
df_clean = df.copy().drop(columns=["Unnamed: 0", "yield", "cook_time", "prep_time", "timing"]).dropna(subset='total_time').reset_index(drop=True)
df_clean.ingredients = df_clean.ingredients.apply(extract_ingredients)
df_clean.nutrition = df_clean.nutrition.apply(extract_nutrition)
df_clean.total_time = df_clean.total_time.apply(extract_time)
df_clean = df_clean.rename(columns={'total_time' : 'total_time_minutes'})
df_clean.directions = df_clean.directions.apply(format_directions)
df_clean["uuid"] = df_clean.apply(lambda _: str(uuid.uuid4()), axis=1)
show_df(df_clean)

(1045, 11)


,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,cuisine_path,nutrition,img_src,uuid
0,Apple Pie by Grandma Ople,90,8,"{'small apples': {'amount': 8.0, 'unit': None}...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,5afae6c3-bc48-4a6a-9289-bf165e50fa5b
1,Sarah's Homemade Applesauce,25,4,"{'apples': {'amount': 4.0, 'unit': None}, 'wat...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,8e46c8d3-75f3-4af8-b56e-d28699a73710
1044,Chef John's Chocolate Energy Bars,180,12,"{'dates': {'amount': 480.0, 'unit': 'ml'}, 'ra...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,/Appetizers and Snacks/Snacks/Granola Bar Reci...,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,60075d5d-9304-4cb6-9943-bc059f918867


## Identification des régimes compatibles et restrictions

In [10]:
meats = [
    "chicken", "beef", "pork", "lamb", "turkey", "duck", "goat",
    "veal", "bacon", "ham", "sausage", "salami", "pepperoni",
    "prosciutto", "venison", "rabbit", "quail", "pheasant",
    "buffalo", "bison", "ostrich", "kangaroo"
]

fish_and_seafood = [
    "salmon", "tuna", "cod", "haddock", "trout", "mackerel",
    "sardines", "anchovies", "halibut", "snapper", "bass",
    "catfish", "tilapia", "flounder", "sole", "grouper",
    "swordfish", "shark", "eel", "herring", "pollock",
    "crab", "lobster", "shrimp", "prawns", "scallops",
    "clams", "mussels", "oysters", "squid", "octopus"
]

plant_based_milks = [
    "almond milk", "almondmilk", "coconut milk", "soy milk", "coconut milk"
]

diet_keywords = {
    "vegan": meats + fish_and_seafood + ["meat", "fish", "dairy", "egg", "butter", "cheese", "milk", "honey", "yogurt"],
    "vegetarian": meats + fish_and_seafood + ["meat", "fish"],
    "pescitarian": meats + ["meat"],
    "omnivore": []
}

allergen_keywords = {
    "eggs": ["egg"],
    "dairy": ["butter", "milk", "cheese", "yogurt"],
    "nuts": ["nuts", "cashew", "almond", "walnut", "pecan", "pistachio", "hazelnut", "macadamia", "brazil nut", "pine nut", "chestnut"],
    "soy": ["soy"]
}

gluten_keywords = ["flour", "wheat", "barley", "rye"]
fodmap_keywords = ["apple", "honey", "garlic", "onion", "wheat"]

def is_plant_based_milk(ingredient):
    return any(re.search(plant_milk, ingredient, re.IGNORECASE) for plant_milk in plant_based_milks)

def identify_diet(ingredients):
    for diet, keywords in diet_keywords.items():
        if all(not any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in keywords) or is_plant_based_milk(ingredient) for ingredient in ingredients):
            return diet
    return "omnivore"

def identify_tags(ingredients):
    tags = []
    for allergen, keywords in allergen_keywords.items():
        if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in keywords) and not is_plant_based_milk(ingredient) for ingredient in ingredients):
            tags.append(f"contains_{allergen}")
    if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in gluten_keywords) for ingredient in ingredients):
        tags.append("contains_gluten")
    if any(any(re.search(keyword, ingredient, re.IGNORECASE) for keyword in fodmap_keywords) for ingredient in ingredients):
        tags.append("high_FODMAP")
    return tags


ingredient_names = df_clean.at[0, 'ingredients'].keys()

diet = identify_diet(ingredient_names)
tags = identify_tags(ingredient_names)
print(f"Diet Compatibility: {diet}")
print(f"Tags: {tags}")


Diet Compatibility: vegetarian
Tags: ['contains_dairy', 'contains_gluten', 'high_FODMAP']


In [11]:
df_clean["diet"] = df_clean.ingredients.apply(identify_diet)
df_clean["tags"] = df_clean.ingredients.apply(identify_tags)
df_clean.cuisine_path = df_clean.cuisine_path.apply(lambda x: x.split('/')[1])
df_clean = df_clean.rename(columns={'cuisine_path' : 'recipe_type'})
show_df(df_clean)

(1045, 13)


,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,recipe_type,nutrition,img_src,uuid,diet,tags
0,Apple Pie by Grandma Ople,90,8,"{'small apples': {'amount': 8.0, 'unit': None}...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,Desserts,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,5afae6c3-bc48-4a6a-9289-bf165e50fa5b,vegetarian,"[contains_dairy, contains_gluten, high_FODMAP]"
1,Sarah's Homemade Applesauce,25,4,"{'apples': {'amount': 4.0, 'unit': None}, 'wat...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,Side Dish,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,8e46c8d3-75f3-4af8-b56e-d28699a73710,vegan,[high_FODMAP]
1044,Chef John's Chocolate Energy Bars,180,12,"{'dates': {'amount': 480.0, 'unit': 'ml'}, 'ra...","Place cashews, almonds, coconut, dates, cocoa,...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,Appetizers and Snacks,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,60075d5d-9304-4cb6-9943-bc059f918867,vegan,[contains_nuts]


In [12]:
df_clean.recipe_type.value_counts()

recipe_type
Desserts                            359
Side Dish                           132
Salad                                90
Drinks Recipes                       82
Cuisine                              68
Appetizers and Snacks                68
Bread                                66
Breakfast and Brunch                 46
Main Dishes                          42
Meat and Poultry                     36
Soups, Stews and Chili Recipes       19
Seafood                              11
Everyday Cooking                     10
Quick Bread Recipes                   3
Sauces and Condiments                 2
Trusted Brands: Recipes and Tips      2
Fruits and Vegetables                 2
BBQ & Grilling                        2
Soup Recipes                          2
Holidays and Events Recipes           2
Mexican                               1
Name: count, dtype: int64

## Traduction anglais > français

In [15]:
import pandas as pd
from deep_translator import GoogleTranslator
from concurrent.futures import ThreadPoolExecutor


def translate_batch(strings):
    return GoogleTranslator(source='en', target='fr').translate_batch(strings)


def translate_dict_batch(dicts):
    keys = [key for d in dicts for key in d.keys()]
    translated_keys = translate_batch(keys)
    translated_dicts = []
    for d in dicts:
        translated_dict = {translated_keys.pop(0): value for key, value in d.items()}
        translated_dicts.append(translated_dict)
    return translated_dicts


def translate_column(df, col):
    if isinstance(df.at[0, col], str):
        df[col] = translate_batch(df[col].tolist())
    elif isinstance(df.at[0, col], dict):
        df[col] = translate_dict_batch(df[col].tolist())
    return df


def translate_df(df: pd.DataFrame, columns=['recipe_name', 'ingredients', 'directions', 'recipe_type', 'nutrition']):
    for col in columns:
        df = translate_column(df, col)
    return df


In [17]:
df_clean_translated = pd.read_csv("/home/camille/code/Camille9999/EPSI/workshop-m1/data/data/generated/recipes_translated.csv", sep=';')
df_clean_translated = df_clean_translated.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
df_clean_translated.nutrition = df_clean.nutrition
df_clean_translated.ingredients = df_clean.ingredients
df_clean_translated.tags = df_clean.tags
df_clean_translated = translate_df(df_clean_translated, columns=['ingredients', 'nutrition'])
show_df(df_clean_translated)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Export du dataframe au format json

In [114]:
result_dict = df_clean.set_index('uuid').to_dict(orient='index')

with open('../data/generated/recipes.json', 'w') as json_file:
    json.dump(result_dict, json_file)

### Format recipes.json

**Clé :** *uuid* de la recette

**Valeurs :**

| Valeur | Nom technique | Format |
|--------|---------------|--------|
| Nom | recipe_name | str |
| Temps de préparation | total_time_minutes | int |
| Parts | servings | int |
| Ingrédients | ingredients | dict |
| Instructions | directions | str |
| Note | rating | float |
| Lien | url | str |
| Nutrition | nutrition | dict |
| Image | img_src | str |
| Régime compatible | diet | str |
| Restrictions alimentaires | tags | list[str] |
| Type de plat | recipe_type | str |

**Format ingrédients :** {
  nom : {
    'amount' : float, 
    'unit' : str | None
  },
  ...
}

**Format nutrition :** { 
  nom : {
    'amount' : str (quantité + unité), 
    'percent' : str (quantité%, % de la dose journalière recommandée)
  },
  ...
}



## Pistes d'amélioration

- Déterminer la saisonalité des recettes en fonction des ingrédients pour privilégier la recommandation de recettes de saison


In [69]:
df_clean_translated = pd.read_csv("/home/camille/code/Camille9999/EPSI/workshop-m1/data/data/generated/recipes_translated.csv", sep=';')
df_clean_translated = df_clean_translated.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
df_clean_translated.nutrition = df_clean.nutrition
show_df(df_clean_translated)

(1045, 13)


,recipe_name,total_time_minutes,servings,ingredients,directions,rating,url,recipe_type,nutrition,img_src,uuid,diet,tags
0,Tarte aux pommes de grand-mère Ople,90,8,"{'petites pommes': {'quantité': 8.0, 'unité': ...","Peler et épépiner les pommes, puis les tranche...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,Desserts,"{'Total Fat': {'amount': '19g', 'percent': '24...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,c4dfd4b8-af04-4a53-9cfd-51b83123555b,vegetarian,"['contains_dairy', 'contains_gluten', 'high_FO..."
1,Compote de pommes maison de Sarah,25,4,"{'pommes': {'quantité': 4,0, 'unité': Aucune},...","Mélangez les pommes, l'eau, le sucre et la can...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,Plat d'accompagnement,"{'Total Fat': {'amount': '0g', 'percent': '0%'...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,4816ee40-3c66-4536-a551-e042b9ab362d,vegan,['high_FODMAP']
1044,Barres énergétiques au chocolat du chef John,180,12,"{'dattes': {'quantité': 480.0, 'unité': 'ml'},...","Mettre les noix de cajou, les amandes, la noix...",4.8,https://www.allrecipes.com/recipe/254452/chef-...,Apéritifs et collations,"{'Total Fat': {'amount': '22g', 'percent': '28...",https://www.allrecipes.com/thmb/IOoj42MMo8YNKW...,52551a0e-cb08-4df0-bc47-25e1e8d3794f,vegan,['contains_nuts']


In [57]:
df_clean_translated.at[0, 'ingredients'].replace("'", '"').replace("Aucune", '"None"')

'{"petites pommes": {"quantité": 8.0, "unité": "None"}, "beurre": {"quantité": 120.0, "unité": "ml"}, "farine tout usage": {"quantité": 44.0, "unité": "ml"}, "sucre blanc": {"quantité": 120.0, "unité": "ml"}, "sucre roux": {"quantité": 120.0, "unité": "ml"}, "eau": {"quantité": 60.0, "unité": "ml"}, "pâte à tarte double croûte": {"quantité": 1.0, "unité": "None"}}'

In [68]:
json.loads(df_clean_translated.at[0, 'nutrition'].replace("'", '"').replace('None', '"None"'))

{'Total Fat': {'amount': '19g', 'percent': '24%'},
 'Saturated Fat': {'amount': '9g', 'percent': '46%'},
 'Cholestérol': {'amount': '31mg', 'percent': '10%'},
 'Sodium': {'amount': '124mg', 'percent': '5%'},
 'Total Carbohydrate': {'amount': '52g', 'percent': '19%'},
 'Dietary Fiber': {'amount': '3g', 'percent': '10%'},
 'Protein': {'amount': '2g', 'percent': 'None'},
 'Vitamine C': {'quantité': '5 mg', 'pourcentage': '27 %'},
 'Calcium': {'quantité': '23 mg', 'pourcentage': '2 %'},
 'Fer': {'quantité': '1 mg', 'pourcentage': '6 %'},
 'Potassium': {'quantité': '156 mg', 'pourcentage': '3 %'}}

In [94]:
ingre =
ingre


['petites pommes',
 'beurre',
 'farine tout usage',
 'sucre blanc',
 'sucre roux',
 'eau',
 'pâte à tarte double croûte']

In [88]:
ingre

["{'petites pommes'",
 "8.0, 'unité': Aucune}, 'beurre'",
 "120.0, 'unité': 'ml'}, 'farine tout usage'",
 "44.0, 'unité': 'ml'}, 'sucre blanc'",
 "120.0, 'unité': 'ml'}, 'sucre roux'",
 "120.0, 'unité': 'ml'}, 'eau'",
 "60.0, 'unité': 'ml'}, 'pâte à tarte double croûte'",
 "1.0, 'unité': Aucune}}"]

In [75]:
import json

def clean_nutri(nutri: str):
    nutri = nutri.replace("'", '"').replace('Aucune', '"None"').replace('quantité', 'amount').replace('unité', 'unit').replace(', ', '; ').replace(',', '.').replace(';', ',').replace('"amount": ', '"amount": "')
    for i in range(10):
        nutri = nutri.replace(f'{i},', f'{i}",')
    try:
        res = json.loads(nutri)
        return res
    except Exception as e:
        print(nutri)
        print(e)


# df_clean_translated.nutrition = df_clean_translated.nutrition.apply(clean_nutri)

for i, row in df_clean_translated.iterrows():
    try:
        clean_nutri(row.ingredients)
    except Exception:
        print(i, row.ingredients.replace("'", '"').replace('Aucune', '"None"').replace('quantité', 'amount').replace('unité', 'unit'))
#show_df(df_clean_translated)


{"pommes tout usage": {"amount": "2400.0", "unit": "ml"}, "sucre blanc": {"amount": "240.0", "unit": "ml"}, "farine tout usage": {"amount": "240.0", "unit": "ml"}, "cannelle moulue": {"amount": "5.0", "unit": "ml"}, "eau": {"amount": "120.0", "unit": "ml"}, "avoine à cuisson rapide": {"amount": "240.0", "unit": "ml"}, "sucre roux": {"amount": "240.0", "unit": "ml"}, "levure chimique": {"amount": "1.0", "unit": "ml"}, "bicarbonate de soude" : {"amount" : 1.0", "unit" : "ml"}, "beurre" : {"amount" : 120.0", "unit" : "ml"}}
Expecting ':' delimiter: line 1 column 444 (char 443)
{"pommes": {"amount": "6.0", "unit": "None"}, "sucre blanc": {"amount": "30.0", "unit": "ml"}, "cannelle moulue": {"amount": "7.0", "unit": "ml"}, "sucre roux": {"amount": "240.0", "unit": "ml"}, "avoine à l"ancienne": {"amount": "180.0", "unit": "ml"}, "farine tout usage": {"amount": "180.0", "unit": "ml"}, "beurre froid": {"amount": "120.0", "unit": "ml"}}
Expecting ':' delimiter: line 1 column 209 (char 208)
{"re

In [ ]:
df_clean_translated.nutrition = df_clean_translated.nutrition.apply(lambda x: json.loads())
df_clean_translated.ingredients = df_clean_translated.ingredients.apply(lambda x: json.loads(x.replace("'", '"').replace('Aucune', '"None"').replace('quantité', 'amount').replace('unité', 'unit')))

show_df(df_clean_translated)